# Processing and Creating Dataframes
#### Creating 4 separate datasets for each region's data, averaging values by month, day, hour and state to reduce size of dataset and consolidate data for analysis

In [ ]:
#importing key libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
#renaming columns
renamed_columns = ['data','hora','precipitacao total,horario (mm)','pressao atmosferica ao nivel da estacao (mb)','pressao atmosferica max. na hora ant. (aut) (mb)','pressao atmosferica min. na hora ant. (aut) (mb)','radiation (kj/m2)','temperatura do ar - bulbo seco (°c)','temperatura do ponto de orvalho (°c)','temperatura maxima na hora ant. (aut) (°c)','temperatura minima na hora ant. (aut) (°c)','temperatura orvalho max. na hora ant. (aut) (°c)','temperatura orvalho min. na hora ant. (aut) (°c)','umidade rel. max. na hora ant. (aut) (%)','umidade rel. min. na hora ant. (aut) (%)','umidade relativa do ar, horaria (%)','vento direcao horaria (gr) (° (gr))','vento rajada maxima (m/s)','vento velocidade horaria (m/s)','region','state','station','station_code','latitude','longitude','height']
renamed_columns_en = ['date','hour','total precipitation (mm)','pressao atmosferica ao nivel da estacao (mb)','atmospheric pressure max. in the previous hour (mb)','atmospheric pressure min. in the previous hour (mb)','radiation (kj/m2)','air temperature - dry bulb (°c)','dew point temperature (°c)','max. temperature in the previous hour (°c)','min. temperature in the previous hour (°c)','dew temperature max. in the previous hour (°c)','dew temperature min. in the previous hour (°c)','relative humidity max. in the previous hour (%)','relative humidity min. in the previous hour (%)','air relative humidity (%)','wind direction (° (gr))','wind rajada maxima (m/s)','wind speed (m/s)','region','state','station','station_code','latitude','longitude','height']
abbreviation = ['date','hour','prcp', 'stp', 'smax', 'smin','gbrd','temp','dewp','tmax','tmin','dmax','dmin','hmax','hmin','hmdy','wdct', 'gust', 'wdsp', 'regi','prov','wsnm','inme','lat','lon','elvt']

def process_raw(df):
    """
    ! drop index
    ! rename the columns
    ! combine hour and date columns -> transform to date_time
    ! append columns data from all stations to keep the data granularity
    ! remove not useful columns
    ! return table 
    """
    df.drop(['index'],inplace=True, axis=1)
    df.columns = renamed_columns_en
    df['date_time'] = pd.to_datetime(df['date'] + ' ' +  df['hour'])
    #df.drop(['date','hour','regi','prov','wsnm','lat','lon','elvt'], inplace=True, axis=1)

    station_codes = list(df.station_code.unique())
    df_models = by_code(df,station_codes[0])
    if len(station_codes)>1:
        for i in station_codes[1:]:
            df_temp = by_code(df,i)
            df_models = pd.concat([df_models,df_temp],axis=1)
    return df_models

def by_code(df, station_code):
    """
    ! function that filters the dataframe by station_code and rename the columns by the status code
    """
    df_station = None
    df_station = df[df['station_code'] == station_code]
    df_station.index = df_station.iloc[:,-1]
    df_station.drop(['station_code','date_time','radiation (kj/m2)'],axis=1,inplace=True)
    df_station.columns = list(map(lambda x: station_code+'_'+x, df_station.columns))
    return df_station

def clean_na(df, na_value = -9999):
    """ 
    ! replace Na from dataset
    """
    df = df.replace(to_replace=na_value,value=np.NaN)
    df = df[df.iloc[:,1].first_valid_index():]
    df = df.fillna(method='ffill')
    return df

def make_dataset(stations, start_date, df):
    """
    ! filter raw data from date and stations code
    ! process data
    ! clean na
    """
    df = df[df['Data'] >= start_date]
    df = df[df['station_code'].apply(lambda x: x in stations)]
    df = process_raw(df)
    df = clean_na(df)
    return df.reset_index()


In [ ]:
from sklearn.metrics import mean_absolute_error, median_absolute_error

def mean_abs_deviation(y):
    return ((y - y.mean()).apply(abs)).mean()

def median_abs_deviation(y):
    return ((y - y.median()).apply(abs)).median()

def create_metrics(y_test,y_pred):
    ret = {}
    ret['mean abs deviation'] = mean_abs_deviation(y_test)
    ret['mean absolute error'] = mean_absolute_error(y_test, y_pred)
    ret['median abs deviation'] = median_abs_deviation(y_test)
    ret['median absolute error'] = median_absolute_error(y_test,y_pred)
    return ret

# Importing and merging datasets

In [ ]:
cw = pd.read_csv("central_west.csv", nrows=10000)
n = pd.read_csv("north.csv", nrows=10000)
ne = pd.read_csv("northeast.csv", nrows=10000)
s = pd.read_csv("south.csv", nrows=10000)
se = pd.read_csv("southeast.csv", nrows=10000)

In [ ]:
df = pd.concat([cw, n, ne, s, se], ignore_index=True)

In [ ]:
df = pd.read_csv("north.csv")

In [ ]:
df.head()

## Processing/Cleaning Data

In [ ]:
#drop index
#rename the columns
#combine hour and date columns -> transform to date_time
#append columns data from all stations to keep the data granularity
#remove not useful columns
#return table 

In [ ]:
df.drop(['index'],inplace=True, axis=1)

In [ ]:
df.columns = renamed_columns_en


In [ ]:
df.rename(columns={'pressao atmosferica ao nivel da estacao (mb)':'atmospheric pressure at station level (mb)'}, inplace=True)

In [ ]:
df.rename(columns={'wind rajada maxima (m/s)':'maximum wind gust (m/s)'}, inplace=True)

In [ ]:
#df.drop(['date','hour','regi','prov','wsnm','lat','lon','elvt'], inplace=True, axis=1)

In [ ]:
df.head()

In [ ]:
df['date_time'] = pd.to_datetime(df['date'] + ' ' +  df['hour'])

In [ ]:
df.tail()

In [ ]:
df.describe(include='all')

In [ ]:
df.info()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
for i in list(df.columns):
    df = df.loc[df[i] != -9999]

In [ ]:
import seaborn as sns
sns.boxplot(data=df['radiation (kj/m2)'], orient='h')
plt.show()

In [ ]:
df.drop(columns=['station', 'station_code'], inplace=True)

In [ ]:
print(list(df.columns))

### Group by month

In [ ]:

renamed = ['date','hour','total_prcp', 'station_pressure', 'pressure_max', 'pressure_min','air_temp','temp','dewp','tmax','tmin','dmax','dmin','hmax','hmin','hmdy','wdct', 'gust', 'wdsp', 'regi','prov','wsnm','inme','lat','lon','elvt']



In [ ]:
month = df.copy()
#month_group = month.groupby(by=[month.date_time.month, month.date_time.year]).agg('mean')
#month_group = month.groupby('state')
#mean_month = month.resample(rule='M', on='date_time').agg('mean')
#mean_month.head()
#month_group.head()

In [ ]:
mean_month.shape

In [ ]:
mean_month.tail()

In [ ]:
mean_month['region'] = 'SE'

In [ ]:
mean_month.head()

In [ ]:
mean_month.to_csv("SE_monthly_rad.csv")

## Group by day

In [ ]:
mean_day = month.resample(rule='D', on='date_time').agg('mean')
mean_day.head()

In [ ]:
mean_day.shape

In [ ]:
mean_day['region'] = 'SE'

In [ ]:
mean_day.to_csv("SE_daily_rad.csv")

## Group by hour

In [ ]:
mean_hour = month.resample(rule='H', on='date_time').agg('mean')
mean_hour.head()

In [ ]:
mean_hour['region'] = 'N'

In [ ]:
mean_hour.shape

In [ ]:
mean_hour.tail()

In [ ]:
mean_hour.to_csv("N_hourly_rad.csv")

## Group by state

In [ ]:
df.state.unique()

In [ ]:
state = df.copy()
state_df = state.groupby('state').agg('mean')
state_df.head()

In [ ]:
state_df.shape

In [ ]:
state_df.to_csv('SE_state_rad.csv')

In [ ]:
#df.to_csv("brazil_weather_data.csv", index=False)